# Simulated PIK statistics

Here we inspect the accuracy and characteristics of the PIKs assigned,
leveraging our knowledge of ground truth from pseudopeople.

It wouldn't be possible to do the ground truth part with the real PVS, but
Layne, Wagner, and Rothhaas did something similar by redacting SSN from real records,
sending them through PVS without the SSN, and then using the true SSN
as ground truth.
The health care records they used are probably quite different from a CUF,
but they found a **very** good overall PIK accuracy (see cell below).

In [1]:
import datetime

from vivarium_research_prl import distributed_compute, utils

In [2]:
print(datetime.datetime.now())

2024-02-12 16:52:09.282716


In [3]:
# DO NOT EDIT if this notebook is not called ground_truth_accuracy.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
simulated_data_output_dir = 'generate_simulated_data/output'
case_study_output_dir = 'output'

# The "compute engine" is what we use on the Python side
# for our case-study-specific operations,
# as opposed to the Splink engine
compute_engine = 'pandas'
# Only matter if using a distributed compute engine
compute_engine_num_jobs = 3
compute_engine_cpus_per_job = 2
compute_engine_memory_per_job = "5GB"

In [4]:
# Parameters
data_to_use = "ri"
simulated_data_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/generate_simulated_data"
case_study_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/person_linkage_case_study"
compute_engine = "dask"
compute_engine_num_jobs = 20
compute_engine_memory_per_job = "30GB"
compute_engine_cpus_per_job = 2


In [5]:
# Parameters for a USA run
# data_to_use = "usa"
# simulated_data_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/generate_simulated_data"
# case_study_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/person_linkage_case_study"

# compute_engine = 'dask'
# compute_engine_num_jobs = 50
# compute_engine_memory_per_job = "120GB"
# compute_engine_cpus_per_job = 2

In [6]:
case_study_output_dir = f'{case_study_output_dir}/{data_to_use}'
simulated_data_output_dir = f'{simulated_data_output_dir}/{data_to_use}'

In [7]:
import os
from pathlib import Path
os.environ["PATH"] = f"{Path('./slurm_within_singularity').resolve()}:{os.environ['PATH']}"

In [8]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=compute_engine_num_jobs,
    cpus_per_job=compute_engine_cpus_per_job,
    memory_per_job=compute_engine_memory_per_job,
)

<Client: 'tcp://10.158.111.17:45491' processes=20 threads=20, memory=558.80 GiB>

In [9]:
census_2030_piked = df_ops.read_parquet(f'{case_study_output_dir}/census_2030_piked.parquet')
confirmed_piks_with_ground_truth = df_ops.read_parquet(f'{case_study_output_dir}/confirmed_piks.parquet')

Imbalanced dataframe: too_few=True, too_many=False, too_large=False
count    3.000000e+00
mean     8.964830e+07
std      1.098194e+05
min      8.952239e+07
25%      8.961028e+07
50%      8.969817e+07
75%      8.971125e+07
max      8.972434e+07
dtype: float64


Imbalanced dataframe: too_few=True, too_many=False, too_large=False
count    2.000000e+00
mean     9.232265e+07
std      9.044807e+06
min      8.592701e+07
25%      8.912483e+07
50%      9.232265e+07
75%      9.552047e+07
max      9.871830e+07
dtype: float64


In [10]:
piked_proportion = df_ops.compute(census_2030_piked.pik.notnull().mean())
# Compare with 90.28% of input records PIKed in the 2010 CUF,
# as reported in Wagner and Layne, Table 2, p. 18 
print(f'{piked_proportion:.2%} of the input records were PIKed')

85.85% of the input records were PIKed


In [11]:
# Multiple Census rows assigned the same PIK, indicating the model thinks they are duplicates in Census
pik_sizes = df_ops.persist(df_ops.groupby_agg_small_groups(census_2030_piked, by='pik', agg_func=lambda x: x.size()))
df_ops.compute(pik_sizes.value_counts())

1    941940
2       477
3         2
Name: count, dtype: int64

In [12]:
# Interesting: in pseudopeople, sometimes siblings are assigned the same (common) first name, making them almost identical.
# The only giveaway is their age and DOB.
# Presumably, this tends not to happen in real life.
duplicate_piks = pik_sizes.rename('pik_size').reset_index().pipe(lambda df: df[df.pik_size > 1])

df_ops.head(census_2030_piked.merge(duplicate_piks, on="pik")).sort_values('pik')

,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,record_id,pik,pik_size
6,8997_599780,<NA>,D,Mungarro,28,02/11/2029,178,squash hllw rd,<NA>,providence,RI,02915,Household,Reference person,Female,Latino,2030,simulated_census_2030_2_47916,12_8442,2
7,8997_599780,Jose,G,Mungarro,1,02/11/2029,<NA>,squash hllw rd,<NA>,providence,RI,02915,Household,Biological child,Male,Latino,2030,simulated_census_2030_2_49403,12_8442,2
0,8612_248111,Amanda,D,Linkerfeldd,51,10/11/1978,463,courvjlle ave,unit 1359,westerly,RI,02860,Household,Opposite-sex spouse,Female,Latino,2030,simulated_census_2030_2_18594,20_16326,2
1,8612_248111,Alana,S,Lingerfeldt,19,10/11/1978,463,courville ave,unit 1359,westerly,RI,02860,Household,Biological child,Female,Latino,2030,simulated_census_2030_2_18595,20_16326,2
8,9225_138217,Cody,J,Wilsoj,42,04/23/1987,40058,corey ave sw,<NA>,providence,RI,02840,Household,Opposite-sex spouse,Male,White,2030,simulated_census_2030_2_61837,51_4315,2
9,9225_138217,Jackson,J,Wilson,11,04/23/1987,40058,corey ave sw,<NA>,providence,RI,02840,Household,Biological child,Male,White,2030,simulated_census_2030_2_61838,51_4315,2
4,8877_33562,Cayden,<NA>,Elmer,9,01/20/2021,382,hill ave,<NA>,lincoln,NaN,02920,Household,Biological child,Male,White,2030,simulated_census_2030_2_43040,70_27612,2
5,8877_33562,Yuna,A,Elmer,9,01/20/2021,382,hill ave,<NA>,lincoln,RI,02920,Household,Biological child,Female,White,2030,simulated_census_2030_2_43044,70_27612,2
2,8641_349693,Husband,U,Judkins,60,07/21/1967,409,red gum st,<NA>,coventry,RI,02857,Household,Reference person,Female,NaN,2030,simulated_census_2030_2_24831,80_11577,2
3,8641_349693,Brian,T,Judkins,62,07/21/1967,409,tred gum st,<NA>,coventry,RI,02857,Household,Opposite-sex spouse,Male,White,2030,simulated_census_2030_2_24832,80_11577,2


## Ground truth statistics

In [13]:
census_2030_ground_truth = df_ops.persist(
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_census_2030_ground_truth.parquet')
)

Imbalanced dataframe: too_few=True, too_many=False, too_large=False
count    3.000000e+00
mean     2.367794e+07
std      1.118165e+07
min      1.076649e+07
25%      2.044768e+07
50%      3.012888e+07
75%      3.013367e+07
max      3.013846e+07
dtype: float64


In [14]:
# In this version of pseudopeople, there are no actual duplicates in Census,
# which means all of the duplicates identified above are wrong.
assert len(census_2030_ground_truth) == len(df_ops.drop_duplicates(census_2030_ground_truth))

Imbalanced dataframe: too_few=True, too_many=False, too_large=False
count    3.000000e+00
mean     2.084089e+07
std      9.848013e+06
min      9.469383e+06
25%      1.799519e+07
50%      2.652099e+07
75%      2.652664e+07
max      2.653229e+07
dtype: float64


In [15]:
reference_files_ground_truth = df_ops.persist(df_ops.concat([
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_geobase_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_name_dob_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
], ignore_index=True))

In [16]:
# However, there can be reference file records that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(reference_files_ground_truth, by='record_id', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1    3105541
2    1552628
3      55256
4       3012
5        156
6          9
7          1
Name: count, dtype: int64

In [17]:
reference_files_ground_truth = df_ops.persist(reference_files_ground_truth.merge(
    n_unique_simulants,
    on='record_id',
    how='left',
))
reference_files_ground_truth.head(n=100)

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_27_14069,8192_1107427,2
1,simulated_geobase_reference_file_27_14069,7016_1077345,2
2,simulated_geobase_reference_file_27_16234,7927_1071261,2
3,simulated_geobase_reference_file_27_16234,3723_616629,2
4,simulated_geobase_reference_file_27_16877,558_1009816,2
...,...,...,...
95,simulated_geobase_reference_file_27_45078,9292_766079,1
96,simulated_geobase_reference_file_27_45085,6487_7497,1
97,simulated_geobase_reference_file_27_45634,5950_742843,2
98,simulated_geobase_reference_file_27_45634,9402_133838,2


In [18]:
df_ops.head(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == df_ops.compute(reference_files_ground_truth.n_unique_simulants.max())])

,record_id,simulant_id,n_unique_simulants
44377,simulated_geobase_reference_file_37_46808,7125_711458,7
44378,simulated_geobase_reference_file_37_46808,7125_711462,7
44379,simulated_geobase_reference_file_37_46808,7125_711454,7
44380,simulated_geobase_reference_file_37_46808,6545_389063,7
44381,simulated_geobase_reference_file_37_46808,7125_711460,7
44382,simulated_geobase_reference_file_37_46808,7125_711463,7
44383,simulated_geobase_reference_file_37_46808,7125_711453,7


In [19]:
census_2030_ground_truth = df_ops.persist(census_2030_ground_truth.merge(
    df_ops.drop_duplicates(reference_files_ground_truth[['simulant_id']]).assign(possible_to_pik=1),
    on='simulant_id',
    how='left',
).assign(possible_to_pik=lambda df: df.possible_to_pik.fillna(0)))
possible_to_pik_proportion = df_ops.compute(census_2030_ground_truth.possible_to_pik.mean())
print(
    f'{(1 - possible_to_pik_proportion):.2%} of the input records are '
    'impossible to PIK correctly, since they are not in any reference files'
)

0.51% of the input records are impossible to PIK correctly, since they are not in any reference files


In [20]:
print(
    f'Assigned PIKs to {(piked_proportion / possible_to_pik_proportion):.2%} of PIK-able records'
)

Assigned PIKs to 86.30% of PIK-able records


In [21]:
reference_file = df_ops.concat([
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_geobase_reference_file.parquet',
    ),
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_name_dob_reference_file.parquet',
    ),
], ignore_index=True)

In [22]:
reference_file_piks = df_ops.persist(reference_file[['record_id', 'pik']])
reference_file_piks

,record_id,pik
npartitions=188,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [23]:
assert len(reference_file_piks) == len(df_ops.drop_duplicates(reference_file_piks[['record_id']]))

In [24]:
pik_simulant_pairs = df_ops.persist(df_ops.drop_duplicates(reference_files_ground_truth.merge(reference_file_piks, on='record_id')[['pik', 'simulant_id']]))

In [25]:
# However, there can be PIKs that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(pik_simulant_pairs, by='pik', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
3     648740
2     622121
5     164197
1     126280
4      84569
7      32508
6      29063
8       8479
9       5805
10      2914
11      1237
12       883
13       404
14       382
16       196
15       184
17       105
18       104
19        75
20        40
22        35
21        33
25        17
23        15
24        14
26        11
27         8
28         6
30         3
29         2
32         2
36         2
34         1
Name: count, dtype: int64

In [26]:
pik_simulant_pairs = df_ops.persist(pik_simulant_pairs.merge(
    n_unique_simulants,
    on='pik',
    how='left',
))
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=188,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [27]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == df_ops.compute(pik_simulant_pairs.n_unique_simulants.max())])

,pik,simulant_id,n_unique_simulants
21650,19_9678,6539_306331,36
21651,19_9678,3167_24580,36
21652,19_9678,5703_612686,36
21653,19_9678,4672_664811,36
21654,19_9678,6654_869908,36
21655,19_9678,6790_78098,36
21656,19_9678,4260_726870,36
21657,19_9678,3304_605514,36
21658,19_9678,7359_424878,36
21659,19_9678,1935_606207,36


## Definitions of accuracy

1. (most strict) Assigning any PIK with multiple simulants is incorrect
2. Assigning a PIK with multiple simulants is neither incorrect nor correct (excluded from denominator)
3. (most lenient) Assigning a PIK with multiple simulants is correct, as long as at least one of those simulants matches the truth

In [28]:
# All modules, Medicare database, calculated from Layne, Wagner, and Rothhaas Table 1 (p. 15)
real_life_pvs_accuracy = 1 - (2_585 + 60_709 + 129_480 + 89_094) / (52_406_981 + 5_170_924 + 49_374_794 + 50_327_034)
f'{real_life_pvs_accuracy:.5%}'

'99.82079%'

### Definition 1

In [29]:
piks_assigned = df_ops.compute(census_2030_piked.pik.notnull().sum())
piks_assigned

942900

In [30]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants > 1])

,pik,simulant_id,n_unique_simulants
0,47_14391,5399_690029,2
1,47_14391,5072_473330,2
2,47_14570,5399_594583,2
3,47_14570,3254_712882,2
4,47_14607,4400_342761,5
5,47_14607,8817_402119,5
6,47_14607,8509_706510,5
7,47_14607,3298_740748,5
8,47_14607,5399_22101,5
11,47_15153,9292_733960,2


In [31]:
single_sim_piks_correct = df_ops.compute(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_piks_correct

64790

In [32]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / piks_assigned
)

0.06871354332378832

In [33]:
assert len(confirmed_piks_with_ground_truth) == piks_assigned

In [34]:
df_ops.head(census_2030_ground_truth.rename(columns={'record_id': 'record_id_census_2030'}))

,record_id_census_2030,simulant_id,possible_to_pik
0,simulated_census_2030_2_29,8425_13861,1.0
1,simulated_census_2030_2_50,8425_25223,1.0
2,simulated_census_2030_2_92,8425_41180,1.0
3,simulated_census_2030_2_262,8425_113842,1.0
4,simulated_census_2030_2_426,8425_181938,1.0
5,simulated_census_2030_2_512,8425_215012,1.0
6,simulated_census_2030_2_551,8425_233178,1.0
7,simulated_census_2030_2_646,8425_273682,1.0
8,simulated_census_2030_2_653,8425_276622,1.0
9,simulated_census_2030_2_857,8425_362226,1.0


In [35]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_correct = df_ops.compute(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_record_links_correct

82564

In [36]:
(
    single_sim_record_links_correct / piks_assigned
)

0.08756389861066921

### Definition 2

In [37]:
single_sim_piks_assigned = len(census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == 1][['pik', 'simulant_id']]))
single_sim_piks_assigned

67014

In [38]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / single_sim_piks_assigned
)

0.9668129047661682

In [39]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_assigned = df_ops.compute(
    (confirmed_piks_with_ground_truth
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .n_unique_simulants == 1).sum()
)
single_sim_record_links_assigned

440042

In [40]:
(
    single_sim_record_links_correct / single_sim_record_links_assigned
)

0.18762754464346584

### Definition 3

In [41]:
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=188,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [42]:
piks_at_least_partially_correct = df_ops.persist(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(df_ops.drop_duplicates)
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id", "pik"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
piks_at_least_partially_correct

,record_id,pik,correct
npartitions=188,,,
,large_string[pyarrow],large_string[pyarrow],bool[pyarrow]
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [43]:
# Overall accuracy, treating it as a black box
piks_correct_proportion = (df_ops.compute(piks_at_least_partially_correct.correct.sum()) / piks_assigned)
piks_correct_proportion

0.5675723830734967

In [44]:
print(f'{piks_correct_proportion:.5%} of the PIKs assigned were correct; compare with {real_life_pvs_accuracy:.5%} in real life')

56.75724% of the PIKs assigned were correct; compare with 99.82079% in real life


In [45]:
# Looking at whether the exact *record* linked was from the same simulant
sim_record_links_at_least_partially_correct = df_ops.persist(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id_raw_input_file", "record_id_reference_file", "pik", "module_name", "pass_name"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
sim_record_links_at_least_partially_correct

,record_id_raw_input_file,record_id_reference_file,pik,module_name,pass_name,correct
npartitions=112,,,,,,
,large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],large_string[pyarrow],bool[pyarrow]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [46]:
len(sim_record_links_at_least_partially_correct)

942900

In [47]:
len(sim_record_links_at_least_partially_correct[['record_id_raw_input_file', 'record_id_reference_file']].drop_duplicates())

942900

In [48]:
(
    df_ops.compute(sim_record_links_at_least_partially_correct.correct.sum()) / piks_assigned
)

0.202855021741436

In [49]:
assert df_ops.compute((df_ops.groupby_agg_small_groups(confirmed_piks_with_ground_truth, by='record_id_raw_input_file', agg_func=lambda x: x.record_id_reference_file.nunique()) <= 1).all())

In [50]:
# Using definition 3 -- at the PIK level
piks_at_least_partially_correct = df_ops.persist(
    piks_at_least_partially_correct
        .rename(columns={'record_id': 'record_id_raw_input_file'})
        .merge(confirmed_piks_with_ground_truth[['record_id_raw_input_file', 'module_name', 'pass_name']], on='record_id_raw_input_file')
)
piks_at_least_partially_correct

,record_id_raw_input_file,pik,correct,module_name,pass_name
npartitions=188,,,,,
,large_string[pyarrow],large_string[pyarrow],bool[pyarrow],large_string[pyarrow],large_string[pyarrow]
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [51]:
# Accuracy by module -- note that this shows the opposite pattern (with the sample data)
# relative to the results of Layne et al., who found GeoSearch was much *more* accurate
df_ops.compute(piks_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
namesearch,0.507764,42955
dobsearch,0.561717,3192
geosearch,0.570005,880505
hhcompsearch,0.595027,16248


In [52]:
# Accuracy by pass -- could be used to tune pass-specific cutoffs, but
# this might not be too informative while we are still using the sample data.
df_ops.compute(piks_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                     
dobsearch    initials name switch                               0.000000   
             first three characters of name                     0.000000   
geosearch    geokey name switch                                 0.500000   
namesearch   DOB and NYSIIS of name                             0.500751   
             year of birth and first two characters of name     0.508850   
hhcompsearch year of birth                                      0.523145   
namesearch   DOB and initials                                   0.523253   
geosearch    house number and street name Soundex               0.559135   
             some name and DOB information                      0.560421   
dobsearch    reverse Soundex of name                            0.561102   
namesearch   birthday and first two characters of name          0.561897   
dobsearch    first two characters of first name and year of...  0.563624   
geosearch    geokey                                             0.573060   
hhcompsearch initials                                           0.612342   

                                                                  size  
module_name  pass_name                                                  
dobsearch    initials name switch                                    2  
             first three characters of name                          1  
geosearch    geokey name switch                                      8  
namesearch   DOB and NYSIIS of name                              31271  
             year of birth and first two characters of name        678  
hhcompsearch year of birth                                        3154  
namesearch   DOB and initials                                     9762  
geosearch    house number and street name Soundex                56278  
             some name and DOB information                      150808  
dobsearch    reverse Soundex of name                              1743  
namesearch   birthday and first two characters of name            1244  
dobsearch    first two characters of first name and year of...    1446  
geosearch    geokey                                             673411  
hhcompsearch initials                                            13094

In [53]:
# Using definition 3 -- at the link level
df_ops.compute(sim_record_links_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
geosearch,0.190662,880505
hhcompsearch,0.191285,16248
namesearch,0.438878,42955
dobsearch,0.448935,3192


In [54]:
df_ops.compute(sim_record_links_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                     
dobsearch    initials name switch                               0.000000   
             first three characters of name                     0.000000   
geosearch    geokey name switch                                 0.125000   
hhcompsearch year of birth                                      0.181674   
geosearch    house number and street name Soundex               0.188759   
             some name and DOB information                      0.188975   
             geokey                                             0.191200   
hhcompsearch initials                                           0.193600   
namesearch   DOB and initials                                   0.430854   
             birthday and first two characters of name          0.433280   
             year of birth and first two characters of name     0.433628   
             DOB and NYSIIS of name                             0.441719   
dobsearch    first two characters of first name and year of...  0.448824   
             reverse Soundex of name                            0.449799   

                                                                  size  
module_name  pass_name                                                  
dobsearch    initials name switch                                    2  
             first three characters of name                          1  
geosearch    geokey name switch                                      8  
hhcompsearch year of birth                                        3154  
geosearch    house number and street name Soundex                56278  
             some name and DOB information                      150808  
             geokey                                             673411  
hhcompsearch initials                                            13094  
namesearch   DOB and initials                                     9762  
             birthday and first two characters of name            1244  
             year of birth and first two characters of name        678  
             DOB and NYSIIS of name                              31271  
dobsearch    first two characters of first name and year of...    1446  
             reverse Soundex of name                              1743

In [55]:
df_ops.compute(sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct].groupby(["module_name", "pass_name"]).size()).sort_values()

module_name   pass_name                                           
dobsearch     first three characters of name                               1
              initials name switch                                         2
geosearch     geokey name switch                                           7
namesearch    year of birth and first two characters of name             384
              birthday and first two characters of name                  705
dobsearch     first two characters of first name and year of birth       797
              reverse Soundex of name                                    959
hhcompsearch  year of birth                                             2581
namesearch    DOB and initials                                          5556
hhcompsearch  initials                                                 10559
namesearch    DOB and NYSIIS of name                                   17458
geosearch     house number and street name Soundex                     45655
         

### Incorrect and missed PIKs

In [56]:
incorrectly_linked_pairs = df_ops.drop_duplicates(
    sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct]
        [["record_id_raw_input_file", "record_id_reference_file"]]
)
incorrectly_linked_pairs

,record_id_raw_input_file,record_id_reference_file
npartitions=112,,
,large_string[pyarrow],large_string[pyarrow]
,...,...
...,...,...
,...,...
,...,...


In [57]:
len(incorrectly_linked_pairs)

751628

In [58]:
comparison_cols = [
    "first_name",
    "middle_name",
    "last_name",
    "date_of_birth",
    "street_number",
    "street_name",
    "unit_number",
    "city",
    "state",
]

incorrect_links = (
    incorrectly_linked_pairs
        .merge(
            census_2030_piked
                .rename(columns={"record_id": "record_id_raw_input_file", "middle_initial": "middle_name"})
                [["record_id_raw_input_file"] + comparison_cols],
            on="record_id_raw_input_file",
            how="left",
        )
        .merge(
            reference_file
                .rename(columns={"record_id": "record_id_reference_file"})
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                [["record_id_reference_file"] + comparison_cols],
            on="record_id_reference_file",
            how="left",
            suffixes=("_census", "_reference_file"),
        )
)
def flatten(xss):
    return [x for xs in xss for x in xs]

df_ops.head(incorrect_links[flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])])

,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,Arman,Arman,N,Nicholas,Durham,Durham,11/12/2002,20021112,22428,<NA>,main street,<NA>,<NA>,<NA>,east greenwich,PROVIDENCE,NaN,RI
1,Wendy,Wendy,C,Crystal,Chi,Chi,10/16/1977,19771016,622,622,deering st,DEERING ST,<NA>,<NA>,blk island,BLK ISLAND,RI,RI
2,Charlee,Charlee,G,Gracie,Baird,Baird,03/06/2028,20280306,207,<NA>,allegheny ave,<NA>,<NA>,<NA>,richmond,<NA>,RI,<NA>
3,Charlene,Charlene,A,Angela,Hood,Hood,07/04/1964,19640704,2683,2683,conover rd,CONOVER RD,<NA>,<NA>,woonsocket,WOONSOCKET,RI,RI
4,Jordan,Jordan,H,Jonathan,Shore,Shore,03/25/2012,20120325,1970,1970,south linwood avenue,SOUTH LINWOOD AVENUE,<NA>,<NA>,north providence,NORTH PROVIDENCE,RI,RI
5,Tzipora,Tzipora,M,Macie,Pierre,Pierre,01/20/2004,20040120,277,277,fairview av,FAIRVIEW AVE,<NA>,<NA>,warwick,WARWICK,RI,RI
6,Brantley,Brantley,M,Mylo,Deshay,Deshay,11/16/2013,20131116,768,768,dunvegan dr,DUNVEGAN DR,<NA>,<NA>,block island,BLOCK ISLAND,RI,RI
7,GBrebnan,Brennan,K,Kaysen,Papadopoulos,Papadopoulos,04/27/2017,20170427,2615,2615,rd n,RD N,<NA>,<NA>,providence,PROVIDENCE,RI,RI
8,Ayden,Ayden,E,Eli,Briggs,Briggs,10/18/2021,20211018,16425,<NA>,piney grove dr,<NA>,<NA>,<NA>,narragansett,<NA>,RI,<NA>
9,Nennjfer,Jennifer,A,Angela,Woodward,Woodward,01/07/1972,19720107,101,101,mcginnis ferry rd,MCGINNIS FERRY RD,<NA>,<NA>,east providence,EAST PROVIDENCE,RI,RI


In [59]:
reference_files_ground_truth

,record_id,simulant_id,n_unique_simulants
npartitions=112,,,
,large_string[pyarrow],large_string[pyarrow],int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [60]:
missed_links = df_ops.persist(
    census_2030_piked[census_2030_piked.pik.isnull()].drop(columns=["pik"])
        .rename(columns={"middle_initial": "middle_name"})
        .merge(census_2030_ground_truth, on="record_id")
        .merge(reference_file.rename(columns=lambda c: c.replace('mailing_address_', '')).merge(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == 1], on="record_id"), on="simulant_id", suffixes=("_census", "_reference_file"))
)

In [61]:
df_ops.compute(census_2030_piked.pik.isnull().sum())

155364

In [62]:
len(missed_links)

271427

In [63]:
simulants_missed = df_ops.head(missed_links[['simulant_id']], n=100).simulant_id.unique()
simulants_missed

<ArrowExtensionArray>
[ '8509_290877',  '8516_542542', '9549_1114486',  '5439_527260',
  '5628_747537',  '5950_169461',  '6324_465458',  '6800_349159',
 '7264_1146002', '7384_1067881',  '7745_524163',  '1609_880974',
  '4621_921183',   '465_658731',   '6991_92179',   '7086_68503',
  '7384_978319',   '778_352858',  '1609_855014',   '1648_44615',
  '3465_579205',  '3481_504676',  '3607_964500',  '8997_534744',
  '5150_248497',  '5781_584065',  '6520_503535', '6539_1182918',
  '7985_937827', '1917_1115761',   '446_319868',   '446_993375',
  '8527_648373',  '8817_710744',   '974_477935',  '9840_563471',
  '5698_601179',  '6519_830648',   '656_675044',   '6606_46389',
  '6701_661155',  '7264_576200',  '7384_875348',   '2298_15984',
  '3793_941611']
Length: 45, dtype: large_string[pyarrow]

In [64]:
for simulant in simulants_missed[0:15]:
    print(simulant)
    display(df_ops.head(missed_links[missed_links.simulant_id == simulant][['simulant_id'] + flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])], n=100))

8509_290877


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,8509_290877,Aufrety,Maha,C,Monique,<NA>,Mitchell,07/04/2006,20050207,2631,702,w main st,RICHLAND,<NA>,<NA>,cumberland,NORTH KINGSTOWN,RI,RI
1,8509_290877,Aufrety,<NA>,C,Cassidy,<NA>,Maloney,07/04/2006,20060704,2631,<NA>,w main st,<NA>,<NA>,<NA>,cumberland,<NA>,RI,<NA>


8516_542542


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
2,8516_542542,Kimberly,Cooper,<NA>,Archie,Lee,Beck,12/29/1984,20210102,907,1616,crescent woods dr,ROCK CREEK VILLA DR,<NA>,<NA>,east providence,PAWTUCKET,RI,RI
3,8516_542542,Kimberly,Kimberly,<NA>,Teri,Lee,Lee,12/29/1984,19841229,907,<NA>,crescent woods dr,<NA>,<NA>,<NA>,east providence,<NA>,RI,<NA>
4,8516_542542,Kimberly,Millie,<NA>,Leah,Lee,Nash,12/29/1984,20271014,907,407,crescent woods dr,GREENBERRY DRIVE,<NA>,<NA>,east providence,COVENTRY,RI,RI


9549_1114486


...

,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
20,4621_921183,Trent,Connor,M,Robert,GNelson,Michel,76/31/2007,20011205,215,<NA>,fm 1387,<NA>,<NA>,<NA>,richmond,<NA>,RI,<NA>


465_658731


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
21,465_658731,Carla,Carla,K,K,Shafer,Shafer,11/22/1973,<NA>,7277,<NA>,north saqe stree,<NA>,<NA>,<NA>,portsmouth,<NA>,RI,<NA>
22,465_658731,Carla,Robert,K,Micah,Shafer,Walton,11/22/1973,19902225,7277,6,north saqe stree,MARAVILLA AVE,<NA>,<NA>,portsmouth,PORTJMOTH,RI,RI


6991_92179


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
23,6991_92179,Ryan,Lucia,H,Jessica,Waite,Lucero,19/13/2000,20100131,691,<NA>,jonesboro road southeast,<NA>,<NA>,<NA>,cranston,<NA>,LA,<NA>
